# HACER REPORTES FOTOGRÁFICOS

In [8]:
#instalar:
#py -m pip install pillow
#py -m pip install openpyxl
#py -m pip install natsort
#py -m pip install tk        

#Esta linea inicia la rutina y todos los comandos necesarios para la ejecución del código    

from PIL import JpegImagePlugin
from decimal import Decimal
import openpyxl
from openpyxl.styles.borders import Border, Side
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils import get_column_letter
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.drawing.spreadsheet_drawing import OneCellAnchor, AnchorMarker 
from openpyxl.drawing.xdr import XDRPoint2D, XDRPositiveSize2D
from openpyxl.utils.units import pixels_to_EMU, cm_to_EMU
JpegImagePlugin._getmp = lambda: None
import glob
import os
from natsort import natsorted


#Esta linea define los bordes de las fotos a ingresar
thick_border = Border(left=Side(style='thick'), 
                     right=Side(style='thick'), 
                     top=Side(style='thick'), 
                     bottom=Side(style='thick'))

half_thick = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thick'), 
                     bottom=Side(style='thick'))

In [9]:
#ESta línea corrobora que PILImage esté correctamente instalado.
print(openpyxl.drawing.image.PILImage)

<module 'PIL.Image' from 'c:\\Users\\gusta\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\PIL\\Image.py'>


In [10]:
#Esta línea permite seleccionar la carpeta en la que se encuentran las fotos ordenadas que se adjuntaran al reporte
import tkinter
import tkinter.filedialog as tkFileDialog

root = tkinter.Tk()
root.lift()
root.attributes('-topmost',True)
root.after_idle(root.attributes,'-topmost',False)
root.withdraw()

folder_path = tkFileDialog.askdirectory()
print(folder_path)
root.destroy()

D:/00.Nubes/OneD2/OneDrive - UNIVERSIDAD NACIONAL DE INGENIERIA/2.Trabajos/1.Torrecom/2023/3. Chancayllo/FOTOS PRIMERA VISITA


In [11]:
#Esta línea selecciona el excel en el que se encuentra el encabezado del reporte
import tkinter
import tkinter.filedialog as tkFileDialog

root = tkinter.Tk()

root.lift()
root.attributes('-topmost',True)
root.after_idle(root.attributes,'-topmost',False)
root.withdraw()
file_path = tkFileDialog.askopenfile()
print(file_path.name)
root.destroy()

D:/00.Nubes/OneD2/OneDrive - UNIVERSIDAD NACIONAL DE INGENIERIA/1.Estudios/01.Proyecto Personal/Encabezado Torrecom - Estructura.xlsx


In [12]:
#En esta parte el usuario deberá indicar:
#1: La letra correspondiente a la columna a la que llega el encabezado del reporte
#2: Lo anchos de cada una de las columnas correspondientes a ese encabezado.

Ultima_letra=input("Indique la última Columna del encabezado en letras ej. 'H':")
column_count =column_index_from_string(Ultima_letra)
column_widths = list() 
for i in range(column_count):
    col_letter = get_column_letter(1+i)
    ancho=input("Indique el ancho de la columna "+col_letter+":")
    ancho_final=Decimal(ancho)+Decimal(0.7109375)
    column_widths.append(ancho_final)
print(*column_widths)


10.7109375 10.7109375 10.7109375 10.7109375 5.7109375 5.7109375 10.7109375 10.7109375 10.7109375 10.7109375 10.7109375 5.7109375 5.7109375


In [6]:

Direccion=folder_path


# columnas de inicio: indica las columnas en las que estarán las imágenes 
columnas_iniciales=[1,8]

#Separar indica la forma en la que se generará el índice de fotos
# con 4 columnas y separar=True:
# 1 - 2 | 5 - 6
# 3 - 4 | 7 - 8
# con 4 columnas y separar=False:
# 1 - 2 | 3 - 4
# 5 - 6 | 7 - 8
# Cualquier otro caso:
# 1 - 2
# 3 - 4
separar=True

# filainicial: indica la fila inicial desde la que se empezarán a colocar imágenes
filainicial=30
# anchoencolumnas: indica el ancho de la imagen en cantidad de columnas
anchoencolumnas=6
# altoenfilas: indica el alto de la imagen en cantida de filas
altoenfilas=1
#tituloenfilas: indica el espacio que hay que dejar par el título de la imagen en cantidad de filas
tituloenfilas=1
#separacionenfilas: indica el espacio en blanco que hay que dejar entre fila y fila
separacionenfilas=1
#TotalSaltoFila: indica el salto en filas entre dos imágenes consecutivas

column_slice=column_widths[(columnas_iniciales[0]-1):(columnas_iniciales[0]+anchoencolumnas-1)]

TotalSaltoFila=altoenfilas+tituloenfilas+separacionenfilas

In [7]:
# insertar multiples imágenes

# create a workbook and grab active worksheet
workbook = openpyxl.load_workbook(file_path.name)
worksheet = workbook.active
p2e = pixels_to_EMU
basewidth=Decimal(sum(column_slice)*Decimal(7.25))
baseheight = Decimal(basewidth)*3/4

CantColumnas=len(columnas_iniciales)

# images list
images = []
anclajes = []
anc_fila=[]
anc_col=[]
insertartitulo=[]
insertarindice=[]
for filename in natsorted(glob.glob(Direccion+"/**/*jpeg",recursive=True)):
    images.append(filename)

#total: Total de imágenes:
total=len(images)

CantFilas=round(total/CantColumnas)

FinalDeFilas=filainicial+TotalSaltoFila*CantFilas+1

#Modificar celdas para que se inserten las imágenes
for row in range(filainicial,FinalDeFilas,TotalSaltoFila):
    for col in columnas_iniciales:
        for filas in range(0,altoenfilas):
            worksheet.row_dimensions[row+filas].height = (baseheight*(Decimal(3/4)))/altoenfilas
        for columnas in range(0,anchoencolumnas):
            col_letter = get_column_letter(col+columnas)
            #worksheet.column_dimensions[col_letter].width = (basewidth*(1/7))/anchoencolumnas
        worksheet.merge_cells(start_row=row,start_column=col,end_row=row+altoenfilas-1,end_column=col+anchoencolumnas-1)
        worksheet.merge_cells(start_row=row+altoenfilas,start_column=col,end_row=row+altoenfilas,end_column=col+anchoencolumnas-2)
                

#Celdas de anclaje
if CantColumnas==4 and separar:
    #primera mitad
    for row in range(filainicial,FinalDeFilas,TotalSaltoFila):
        for col in columnas_iniciales[:2]:
            col_letter = get_column_letter(col)
            anclaje = col_letter + str(row)
            anc_fila.append(row)
            anc_col.append(col)
            anclajes.append(anclaje)
    #segunda mitad
    for row in range(filainicial,FinalDeFilas,TotalSaltoFila):
        for col in columnas_iniciales[2:]:
            col_letter = get_column_letter(col)
            anclaje = col_letter + str(row)
            anc_fila.append(row)
            anc_col.append(col)
            anclajes.append(anclaje)    
else:
    #todoo de golpe
    for row in range(filainicial,FinalDeFilas,TotalSaltoFila):
        for col in columnas_iniciales:
            col_letter = get_column_letter(col)
            anclaje = col_letter + str(row)
            anc_fila.append(row)
            anc_col.append(col)
            anclajes.append(anclaje)

# insertar imagenes
for index, image in enumerate(images):
    img=Image(image)
    hpercent = (float(baseheight) / float(img.height))
    wsize = int((float(img.width) * float(hpercent)))
    coloff=0
    filoff=0
    if wsize>basewidth:
        wpercent = (float(basewidth) / float(img.width))
        hsize = int((float(img.height) * float(wpercent)))
        img.width=basewidth
        img.height=hsize
        filoff=p2e((baseheight-hsize)/2)

    else:
        img.width=wsize
        img.height=baseheight
        coloff=p2e((basewidth-wsize)/2)

    img.quality=50
    h, w = img.height, img.width
    size = XDRPositiveSize2D(p2e(w), p2e(h))
    marker = AnchorMarker(col=anc_col[index]-1, colOff=coloff, row=anc_fila[index]-1, rowOff=filoff)
    img.anchor = OneCellAnchor(_from=marker, ext=size)
    worksheet.add_image(img)
    #print(index,image)

if tituloenfilas==1:
    titles = []
    #obtener títulos de los nombres de las carpetas
    for title in natsorted(glob.glob(Direccion+"/**/*jpeg",recursive=True)):
        titles.append(os.path.basename(os.path.dirname(title)))
    
    #escribir títulos debajo de las celdas
    for index, title in enumerate(titles):

        #worksheet.cell(row=anc_fila[index], column=anc_col[index])
        #worksheet.cell(row=anc_fila[index], column=anc_col[index]+1)
        for filas in range(altoenfilas+1):
            for columnas in range(anchoencolumnas):
                worksheet.cell(row=anc_fila[index]+filas, column=anc_col[index]+columnas).border=thick_border

        worksheet.cell(row=anc_fila[index]+1, column=anc_col[index], value=title)
        if anchoencolumnas>1:
            worksheet.cell(row=anc_fila[index]+1, column=anc_col[index]+anchoencolumnas-1, value=index+1)


 
# save workbook

workbook.save(Direccion+'/multiple_images.xlsx')
workbook.close()